In [8]:
import os
import sys
import math
import operator
import random
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
import xgboost as xgb
from scipy import sparse
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
color = sns.color_palette()

%matplotlib inline


In [2]:
#data preprocessing happens here:

class PreProcessing(object):

    def __init__(self, train_df, test_df):
            train_df, test_df=self.vectorizeFeatures(train_df, test_df)
            train_df, test_df=self.labelCategorical(train_df,test_df)
    
    #replacing redundant features in features column
    def repalceRedundantFeatures(self,X):
        X['features']=X['features'].str.replace("garden/patio","garden")
        X['features']=X['features'].str.replace("patio","garden")
        X['features']=X['features'].str.replace("residents_garden","garden")
        X['features']=X['features'].str.replace("common_garden","garden")

        X['features']=X['features'].str.replace("wifi_access","wifi")

        X['features']=X['features'].str.replace("24/7","24")
        X['features']=X['features'].str.replace("24-hour","24")
        X['features']=X['features'].str.replace("24hr","24")
        X['features']=X['features'].str.replace("concierge","doorman")
        X['features']=X['features'].str.replace("ft_doorman","doorman")
        X['features']=X['features'].str.replace("24_doorman","doorman")
        X['features']=X['features'].str.replace("24_hr_doorman","doorman")
        X['features']=X['features'].str.replace("doorman_service","doorman")
        X['features']=X['features'].str.replace("full-time_doorman","doorman")

        X['features']=X['features'].str.replace("gym/fitness","fitness")
        X['features']=X['features'].str.replace("fitness_room","fitness")

        X['features']=X['features'].str.replace("washer","laundry")
        X['features']=X['features'].str.replace("laundry_in_bldg","laundry")
        X['features']=X['features'].str.replace("laundry_in_building","laundry")
        X['features']=X['features'].str.replace("laundry_in_building/dryer","laundry")
        X['features']=X['features'].str.replace("laundry_in_building_&_dryer","laundry")
        X['features']=X['features'].str.replace("laundry_room","laundry")
        X['features']=X['features'].str.replace("laundry_&_housekeeping","laundry")
        X['features']=X['features'].str.replace("laundry_in_unit","laundry")
        X['features']=X['features'].str.replace("laundry_in-unit","laundry")
        X['features']=X['features'].str.replace("laundry_on_every_floor","laundry")
        X['features']=X['features'].str.replace("laundry_on_floor","laundry")
        X['features']=X['features'].str.replace("in-unit_laundry/dryer","laundry")
        X['features']=X['features'].str.replace("on-site_laundry","laundry")
        X['features']=X['features'].str.replace("laundry/dryer","laundry")

        X['features']=X['features'].str.replace("high-speed_internet","high_speed_internet")
        X['features']=X['features'].str.replace("high_speed_internet_available","high_speed_internet")
#------------------------
        X['features']=X['features'].str.replace("parking_available","parking")
        X['features']=X['features'].str.replace("parking_space","parking")
        X['features']=X['features'].str.replace("on-site_garage","parking")
        X['features']=X['features'].str.replace("on-site_parking","parking")
        X['features']=X['features'].str.replace("on-site_parking_lot","parking")
        X['features']=X['features'].str.replace("full_service_garage","parking")
        X['features']=X['features'].str.replace("common_parking/garage","parking")
        X['features']=X['features'].str.replace("garage","parking")
        X['features']=X['features'].str.replace("assigned-parking-space","private_parking")

        X['features']=X['features'].str.replace("storage_available","storage")
        X['features']=X['features'].str.replace("storage_facilities_available","storage")
        X['features']=X['features'].str.replace("storage_space","storage")
        X['features']=X['features'].str.replace("storage_room","storage")
        X['features']=X['features'].str.replace("common_storage","storage")

        X['features']=X['features'].str.replace("central_a/c","central_air")
        X['features']=X['features'].str.replace("central_ac","central_air")
        X['features']=X['features'].str.replace("air_conditioning","central_air")
    
        X['features']=X['features'].str.replace("close_to_subway","subway")

        X['features']=X['features'].str.replace("roofdeck","roof_deck")
        X['features']=X['features'].str.replace("roof-deck","roof_deck")
        X['features']=X['features'].str.replace("rooftop_terrace","roof_deck")
        X['features']=X['features'].str.replace("rooftop_deck","roof_deck")
        X['features']=X['features'].str.replace("roof_access","roof_deck")
        X['features']=X['features'].str.replace("common_roof_deck","roof_deck")
        
        X['features']=X['features'].str.replace("swimming_pool","pool")
        X['features']=X['features'].str.replace("indoor_pool","pool")
        
        X['features']=X['features'].str.replace("deco_fireplace","fireplaces")
        X['features']=X['features'].str.replace("decorative_fireplace","fireplaces")

        return X
    # Using label Encoder to encode Categorical Features
    
    def labelCategorical(self,train_df,test_df):
        categorical = ["display_address", "manager_id", "building_id", "street_address"]
        for f in categorical:
            if train_df[f].dtype=='object':
                lbl = preprocessing.LabelEncoder()
                lbl.fit(list(train_df[f].values) + list(test_df[f].values))
                train_df[f] = lbl.transform(list(train_df[f].values))
                test_df[f] = lbl.transform(list(test_df[f].values))
        return train_df,test_df
            
    # Using Count Vectorizer
    def vectorizeFeatures(self,train_df,test_df):
        train_df['features'] = train_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
        test_df['features'] = test_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
        train_df=self.repalceRedundantFeatures(train_df)
        test_df=self.repalceRedundantFeatures(test_df)
        tfidf = CountVectorizer(stop_words='english', max_features=200)
        global tr_sparse
        tr_sparse = tfidf.fit_transform(train_df["features"])
        global te_sparse
        te_sparse = tfidf.transform(test_df["features"])
        return train_df,test_df

In [3]:
#Feature Engineering Class:
#Contains member functions used to create new features from existing columns in the dataset.

class FeatureEngineering(object):
    
    def __init__(self, train_df, test_df):
        train_df, test_df=self.dateTimeFeatures(train_df, test_df)
        train_df, test_df=self.sinCosHrMnth(train_df, test_df)
        train_df, test_df=self.locationLabel(train_df, test_df)
        train_df, test_df=self.roomFeatures(train_df, test_df)
        train_df, test_df=self.numericalFeatures(train_df, test_df)
        train_df, test_df=self.managerFeatures(train_df,test_df)
        
#creating date time features from created feature
    def dateTimeFeatures(self,train_df,test_df):
        train_df["created"]=pd.to_datetime(train_df["created"])
        test_df["created"]=pd.to_datetime(test_df["created"])


        train_df["created_year"]=train_df["created"].dt.year
        test_df["created_year"]=test_df["created"].dt.year

        train_df["created_month"]=train_df["created"].dt.month
        test_df["created_month"]=test_df["created"].dt.month

        train_df["created_day"]=train_df["created"].dt.day
        test_df["created_day"]=test_df["created"].dt.day

        train_df["created_hour"] = train_df["created"].dt.hour
        test_df["created_hour"] = test_df["created"].dt.hour
        return train_df,test_df
    
    #scaling hour and month using numpy cos and sin
    def sinCosHrMnth(self,train_df,test_df):
        train_df['hr_sin'] = np.sin(train_df.created_hour*(2.*np.pi/24))
        train_df['hr_cos'] = np.cos(train_df.created_hour*(2.*np.pi/24))
        train_df['mnth_sin'] = np.sin((train_df.created_month-1)*(2.*np.pi/12))
        train_df['mnth_cos'] = np.cos((train_df.created_month-1)*(2.*np.pi/12))

        test_df['hr_sin'] = np.sin(test_df.created_hour*(2.*np.pi/24))
        test_df['hr_cos'] = np.cos(test_df.created_hour*(2.*np.pi/24))
        test_df['mnth_sin'] = np.sin((test_df.created_month-1)*(2.*np.pi/12))
        test_df['mnth_cos'] = np.cos((test_df.created_month-1)*(2.*np.pi/12))
        return train_df,test_df
    
    #clustering locations using latitude and longitude
    def locationLabel(self,train_df,test_df):
        loc=train_df.loc[:,['latitude','longitude']]
        loc_=test_df.loc[:,['latitude','longitude']]
        kmeans = KMeans(n_clusters=10, random_state=0)
        kmeans.fit(loc)
        train_df["loc_label"]=kmeans.labels_
        kmeans.fit(loc_)
        test_df["loc_label"]=kmeans.labels_
        return train_df,test_df
        
    #defining numerical features according to their lengths
    def numericalFeatures(self,train_df,test_df):
        train_df["num_photos"]=train_df["photos"].apply(len)
        test_df["num_photos"] = test_df["photos"].apply(len)

        train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
        test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

        train_df["num_features"]=train_df["features"].apply(len)
        test_df["num_features"]=test_df["features"].apply(len)
        return train_df,test_df
    
    #defining price per room
    def roomFeatures(self,train_df,test_df):
        train_df["price_t"] =train_df["price"]/train_df["bedrooms"]
        test_df["price_t"] = test_df["price"]/test_df["bedrooms"] 

        train_df['room_sum'] = train_df['bedrooms']  + train_df['bathrooms']
        test_df['room_sum'] = test_df['bedrooms']  + test_df['bathrooms']

        train_df['price_per_room'] = train_df['price']/train_df['room_sum']
        test_df['price_per_room'] = test_df['price']/test_df['room_sum']
        return train_df,test_df
    
    #creating manager level columns
    def managerFeatures(self,train_df,test_df):
        #manager
        index=list(range(train_df.shape[0]))
        random.shuffle(index)
        a=[np.nan]*len(train_df)
        b=[np.nan]*len(train_df)
        c=[np.nan]*len(train_df)

        for i in range(5):
            building_level={}
            for j in train_df['manager_id'].values:
                building_level[j]=[0,0,0]
            test_index=index[int((i*train_df.shape[0])/5):int(((i+1)*train_df.shape[0])/5)]
            train_index=list(set(index).difference(test_index))
            for j in train_index:
                temp=train_df.iloc[j]
                if temp['interest_level']=='low':
                    building_level[temp['manager_id']][0]+=1
                if temp['interest_level']=='medium':
                    building_level[temp['manager_id']][1]+=1
                if temp['interest_level']=='high':
                    building_level[temp['manager_id']][2]+=1
            for j in test_index:
                temp=train_df.iloc[j]
                if sum(building_level[temp['manager_id']])!=0:
                    a[j]=building_level[temp['manager_id']][0]*1.0/sum(building_level[temp['manager_id']])
                    b[j]=building_level[temp['manager_id']][1]*1.0/sum(building_level[temp['manager_id']])
                    c[j]=building_level[temp['manager_id']][2]*1.0/sum(building_level[temp['manager_id']])
        train_df['manager_level_low']=a
        train_df['manager_level_medium']=b
        train_df['manager_level_high']=c
        a=[]
        b=[]
        c=[]
        building_level={}
        for j in train_df['manager_id'].values:
            building_level[j]=[0,0,0]
        for j in range(train_df.shape[0]):
            temp=train_df.iloc[j]
            if temp['interest_level']=='low':
                building_level[temp['manager_id']][0]+=1
            if temp['interest_level']=='medium':
                building_level[temp['manager_id']][1]+=1
            if temp['interest_level']=='high':
                building_level[temp['manager_id']][2]+=1

        for i in test_df['manager_id'].values:
            if i not in building_level.keys():
                a.append(np.nan)
                b.append(np.nan)
                c.append(np.nan)
            else:
                a.append(building_level[i][0]*1.0/sum(building_level[i]))
                b.append(building_level[i][1]*1.0/sum(building_level[i]))
                c.append(building_level[i][2]*1.0/sum(building_level[i]))
        test_df['manager_level_low']=a
        test_df['manager_level_medium']=b
        test_df['manager_level_high']=c

        return train_df,test_df

In [4]:
#using various models
class Model(object):
    
    #random forest classifier
    def trainRandomForest(self,train_data,train_labels):
        numeric_feats=["bathrooms","bedrooms","price","num_photos","num_description_words","num_features"]
        X_train=train_data[numeric_feats]
        y_train=train_labels
        clf=RandomForestClassifier(n_estimators=1000)
        clf.fit(X_train,y_train)
        return clf
    
    def testRandomForest(self,clf,X_test):
        numeric_feats=["bathrooms","bedrooms","price","num_photos","num_description_words","num_features"]
        test_num=X_test[numeric_feats]
        y_test_pred=clf.predict_proba(test_num)
        return y_test_pred
    
    #using xgboost
    def trainXGB(self, train_X, train_y, seed_val=321, num_rounds=1800):
        param = {}
        param['objective'] = 'multi:softprob'
        param['eta'] = 0.02
        param['max_depth'] = 6
        param['silent'] = 1
        param['num_class'] = 3
        param['eval_metric'] = "mlogloss"
        param['min_child_weight'] = 1
        param['subsample'] = 0.7
        param['colsample_bytree'] = 0.7
        param['seed'] = seed_val
        num_rounds = num_rounds

        plst = list(param.items())
        xgtrain = xgb.DMatrix(train_X, label=train_y)

        #xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

        #pred_test_y = model.predict(xgtest)
        return model

    def testXGB(self, test_X, clf, seed_val=321, num_rounds=1800):
        param = {}
        param['objective'] = 'multi:softprob'
        param['eta'] = 0.02
        param['max_depth'] = 6
        param['silent'] = 1
        param['num_class'] = 3
        param['eval_metric'] = "mlogloss"
        param['min_child_weight'] = 1
        param['subsample'] = 0.7
        param['colsample_bytree'] = 0.7
        param['seed'] = seed_val
        num_rounds = num_rounds

        xgtest = xgb.DMatrix(test_X)
        pred_test_y = clf.predict(xgtest)
        
        return pred_test_y

    

In [5]:
#main class
class RentalListing(object):
    
    def __init__(self, trainFile, testFile):
        self.trainFile = trainFile
        self.testFile = testFile
        self.__model=Model()

    #reading train data
    def trainingData(self):
        self.train_data = pd.read_json(self.trainFile)
        self.train_labels = self.train_data['interest_level']
        target_num_map = {'high':0, 'medium':1, 'low':2}
        self.train_mlabels = np.array(self.train_data['interest_level'].apply(lambda x: target_num_map[x]))
        
    #reading test data
    def testingData(self):
        self.test_data= pd.read_json(self.testFile)
        #self.test_labels = self.test_data['interest_level']
        #target_num_map = {'high':0, 'medium':1, 'low':2}
        #self.test_mlabels = np.array(self.test_data['interest_level'].apply(lambda x: target_num_map[x]))
        
    def data(self):
        self.trainingData()
        self.testingData()
    
    #calling various models
    
    def trainRandomForest(self):
        randomForestClf = self.__model.trainRandomForest(self.train_data,self.train_labels)
        return randomForestClf
    
    def testRandomForest(self,clf):
        return self.__model.testRandomForest(clf,self.test_data)
        
    def trainXGB(self,features_to_use):
            train_X = sparse.hstack([self.train_data[features_to_use], tr_sparse]).tocsr()
            return self.__model.trainXGB(train_X,self.train_mlabels)
            
    def testXGB(self,features_to_use,clf):
            test_X = sparse.hstack([self.test_data[features_to_use], te_sparse]).tocsr()
            return self.__model.testXGB(test_X,clf)

In [6]:
if __name__ == "__main__":
    train_data_name = 'train.json'
    test_data_name = 'test.json'

    #Make a main class object and load data-------------------------------------------------------------------------- 
    rentalListing = RentalListing(train_data_name,test_data_name)
    rentalListing.data()

    # Do Feature Engineering and Preprocessing before running Models
    FeatureEngineering(rentalListing.train_data,rentalListing.test_data)
    PreProcessing(rentalListing.train_data,rentalListing.test_data)
    
    #------------------------------------------------------------------------------------------------------------------
    #train XGB using these features:
    
    numeric_feats=["bathrooms", "bedrooms", "latitude", "longitude","loc_label", "price","price_t","price_per_room", 
    "num_photos", "num_features", "num_description_words","listing_id", 
    "created_year","created_month" , "created_day",'created_hour', ]

    categorical = ["display_address", "manager_id", "building_id", "street_address"]
    features_to_use=numeric_feats + categorical
    
    #------------------------------------------------
    #train XGB
    
    #clf_xg=rentalListing.trainXGB(features_to_use)
    
    #test XGB
    
    #xgb_preds=rentalListing.trainXGB(features_to_use,clf_xg)
    
    #------------------------------------------------
    #if pickle file is to be loaded:
    #loaded_model=pickle.load(open('xgb_model_pickle','rb'))
    
    #test XGB using pickle's loaded model:
    
    #xgb_preds=rentalListing.trainXGB(features_to_use,loaded_model)
    
    #--------------------------------------------------
    
    #Calculate Log Loss:
    #here val_y is the target label in test data
        #val_y=rentalListing.test_mlabels
    #print(log_loss(val_y, xgb_preds))
    
    #--------------------------------------------------
    
    #To run Random Forest-------------------------------------------------------------------------
    
    #train------------------------------------------- 
    #clf_rf=rentalListing.trainRandomForest()
    
    #test using clf returned by training--------------------
    #preds_rf= rentalListing.testRandomForest(clf_rf)
    
    #Calculate Log loss------------------------------------------
    #here val_y is the target label in test data
        #val_y=rentalListing.test_labels
    #print(log_loss(val_y,preds_rf)